In [0]:
def clean_string(string1):
  char1 = "["
  char2 = "]"
  mystr = string1
  return mystr[mystr.find(char1)+1 : mystr.find(char2)]

def clean_commas(string1):
  string1 = string1[1:]
  string1 = string1[:-1]
  return string1

def clean(string1):
  string1 = clean_string(string1)
  string1 = clean_commas(string1)
  return string1

def clean_price(string1):
  string1 = clean_string(string1)
  string1 = clean_commas(string1)
  string1 = string1.split(' ')[1]
  string1 = string1.replace(',', '.', 1)
  return string1

def comma_dot(string1):
  string1 = string1.replace(',', '.', 1)
  return string1

def delete_n(string1):
  string1 = string1.replace('\n', '')
  return string1

In [0]:
#ci permette di importare qualsiasi modulo
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/TensorFlow 2.0/modules')
import pandas as pd
import tf_dataset_extractor as e
#import grapher_v1_1 as g
#import LSTM_creator_v1_0 as l
v = e.v
g = e.g
l = e.l

In [59]:
import pandas as pd
X = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Projects/20200612_Wine_Price_Estimator/wines_.csv')
X = X.drop(['Unnamed: 0', '8', '9', '5'], axis=1)
X

,0,1,2,3,4,6,7,Unnamed: 11
0,['Cantine Ceci'],['Red bubbly'],['N/A'],['\nIGT Emilia o dell’Emilia\n'],['Emilia-Romagna'],['11.00% by volume'],"['0,75 l Standard']",['Price\xa0€ 11.05']
1,['Tenute Rubino'],['Red still'],['2017'],['\nIGT Salento\n'],['Apulia'],['13.00% by volume'],"['0,75 l Standard']",['Price\xa0€ 11.31']
2,['Rocche Malatestiane'],['Red still'],['2017'],['\nDOC Sangiovese di Romagna Superiore\n'],['Emilia-Romagna'],['13.50% by volume'],"['0,75 l Standard']",['Price\xa0€ 10.91']
3,['Tommasi'],['Red still'],['2016'],['\nDOC Valpolicella Ripasso Classico Superior...,['Veneto'],['13.00% by volume'],"['0,75 l Standard']",['Price\xa0€ 11.96']
4,['Jacopo Biondi Santi'],['Red still'],['2012'],['\nIGT Toscano o Toscana\n'],['Tuscany'],['13.50% by volume'],"['0,75 l Standard']",['€ 16.47']
...,...,...,...,...,...,...,...,...
970,['Tenuta La Fuga (Folonari)'],['Red still'],['2015'],"['\n', '\n']",['Tuscany'],['14.50% by volume'],"['0,75 l Standard']",['€ 34.16']
971,['Castelvecchio (Famiglia Terraneo)'],['Red still'],['2018'],['\nDOC Carso Terrano\n'],['Friuli-Venezia Giulia'],['12.50% by volume'],"['0,75 l Standard']",['Price\xa0€ 14.19']
972,['Nicodemi'],['Red still'],['2018'],"[""\nDOCG Montepulciano d'Abruzzo Colline Teram...",['Abruzzo'],['13.50% by volume'],"['0,75 l Standard']",['Price\xa0€ 17.36']
973,['Allegrini'],['Red still'],['2019'],['\nDOC Bardolino\n'],['Veneto'],['12.50% by volume'],"['0,75 l Standard']",['Price\xa0€ 8.59']


In [60]:
X['0'] = X['0'].apply(lambda x : clean(x))
X['1'] = X['1'].apply(lambda x : clean(x))
X['2'] = X['2'].apply(lambda x : clean(x))
X['3'] = X['3'].apply(lambda x : clean(x))
X['3'] = X['3'].apply(lambda x : delete_n(x))
X['4'] = X['4'].apply(lambda x : clean(x))
X['6'] = X['6'].apply(lambda x : clean(x))
X['7'] = X['7'].apply(lambda x : clean(x))
X['Unnamed: 11'] = X['Unnamed: 11'].apply(lambda x : clean_price(x))
X['Unnamed: 11'] = X['Unnamed: 11'].apply(lambda x : float(x))
X

,0,1,2,3,4,6,7,Unnamed: 11
0,Cantine Ceci,Red bubbly,N/A,\nIGT Emilia o dell’Emilia\n,Emilia-Romagna,11.00% by volume,"0,75 l Standard",11.05
1,Tenute Rubino,Red still,2017,\nIGT Salento\n,Apulia,13.00% by volume,"0,75 l Standard",11.31
2,Rocche Malatestiane,Red still,2017,\nDOC Sangiovese di Romagna Superiore\n,Emilia-Romagna,13.50% by volume,"0,75 l Standard",10.91
3,Tommasi,Red still,2016,\nDOC Valpolicella Ripasso Classico Superiore\n,Veneto,13.00% by volume,"0,75 l Standard",11.96
4,Jacopo Biondi Santi,Red still,2012,\nIGT Toscano o Toscana\n,Tuscany,13.50% by volume,"0,75 l Standard",16.47
...,...,...,...,...,...,...,...,...
970,Tenuta La Fuga (Folonari),Red still,2015,"\n', '\n",Tuscany,14.50% by volume,"0,75 l Standard",34.16
971,Castelvecchio (Famiglia Terraneo),Red still,2018,\nDOC Carso Terrano\n,Friuli-Venezia Giulia,12.50% by volume,"0,75 l Standard",14.19
972,Nicodemi,Red still,2018,\nDOCG Montepulciano d'Abruzzo Colline Teramane\n,Abruzzo,13.50% by volume,"0,75 l Standard",17.36
973,Allegrini,Red still,2019,\nDOC Bardolino\n,Veneto,12.50% by volume,"0,75 l Standard",8.59


In [61]:
X.columns = ['Producer', 'Kind', 'Year', 'Location', 'Region', 'Alchohol', 'Volume', 'Price']
X

,Producer,Kind,Year,Location,Region,Alchohol,Volume,Price
0,Cantine Ceci,Red bubbly,N/A,\nIGT Emilia o dell’Emilia\n,Emilia-Romagna,11.00% by volume,"0,75 l Standard",11.05
1,Tenute Rubino,Red still,2017,\nIGT Salento\n,Apulia,13.00% by volume,"0,75 l Standard",11.31
2,Rocche Malatestiane,Red still,2017,\nDOC Sangiovese di Romagna Superiore\n,Emilia-Romagna,13.50% by volume,"0,75 l Standard",10.91
3,Tommasi,Red still,2016,\nDOC Valpolicella Ripasso Classico Superiore\n,Veneto,13.00% by volume,"0,75 l Standard",11.96
4,Jacopo Biondi Santi,Red still,2012,\nIGT Toscano o Toscana\n,Tuscany,13.50% by volume,"0,75 l Standard",16.47
...,...,...,...,...,...,...,...,...
970,Tenuta La Fuga (Folonari),Red still,2015,"\n', '\n",Tuscany,14.50% by volume,"0,75 l Standard",34.16
971,Castelvecchio (Famiglia Terraneo),Red still,2018,\nDOC Carso Terrano\n,Friuli-Venezia Giulia,12.50% by volume,"0,75 l Standard",14.19
972,Nicodemi,Red still,2018,\nDOCG Montepulciano d'Abruzzo Colline Teramane\n,Abruzzo,13.50% by volume,"0,75 l Standard",17.36
973,Allegrini,Red still,2019,\nDOC Bardolino\n,Veneto,12.50% by volume,"0,75 l Standard",8.59


In [62]:
#drop N/A column in Year
X = X[~X['Year'].isin(['N/A'])]

#drop "\\n', '\\n" column in Location
X = X[~X['Location'].isin(["\\n', '\\n"])]

X['Alchohol'] = X['Alchohol'].apply(lambda x : x[0:4])
X['Alchohol'] = X['Alchohol'].apply(lambda x : float(comma_dot(x)))
X

,Producer,Kind,Year,Location,Region,Alchohol,Volume,Price
1,Tenute Rubino,Red still,2017,\nIGT Salento\n,Apulia,13.0,"0,75 l Standard",11.31
2,Rocche Malatestiane,Red still,2017,\nDOC Sangiovese di Romagna Superiore\n,Emilia-Romagna,13.5,"0,75 l Standard",10.91
3,Tommasi,Red still,2016,\nDOC Valpolicella Ripasso Classico Superiore\n,Veneto,13.0,"0,75 l Standard",11.96
4,Jacopo Biondi Santi,Red still,2012,\nIGT Toscano o Toscana\n,Tuscany,13.5,"0,75 l Standard",16.47
5,Cantine Leonardo da Vinci,Red still,2018,\nDOCG Chianti\n,Tuscany,13.0,"0,75 l Standard",9.04
...,...,...,...,...,...,...,...,...
969,Fento Wines,Red organic still,2015,\nDO Rias Baixas\n,Galicia,11.5,"0,75 l Standard",13.14
971,Castelvecchio (Famiglia Terraneo),Red still,2018,\nDOC Carso Terrano\n,Friuli-Venezia Giulia,12.5,"0,75 l Standard",14.19
972,Nicodemi,Red still,2018,\nDOCG Montepulciano d'Abruzzo Colline Teramane\n,Abruzzo,13.5,"0,75 l Standard",17.36
973,Allegrini,Red still,2019,\nDOC Bardolino\n,Veneto,12.5,"0,75 l Standard",8.59


In [63]:
e.X = X
v.partition.one_hot(['Location', 'Producer', 'Kind', 'Year', 'Region','Volume'])
e.X

,Alchohol,Price,Location_ \nDOCG Valtellina Superiore inferno\n,Location_ \nIGT Rubicone\n,Location_\n DO Bierzo\n,Location_\n DOC Primitivo di Manduria\n,Location_\n DOC Romagna Sangiovese Superiore\n,Location_\n DOCG Chianti Classico\n,Location_\n DOCG Chianti\n,Location_\n DOCG Vino Nobile di Montepulciano\n,Location_\n IGT Puglia\n,Location_\n IGT Terre Siciliane\n,Location_\n IGT Toscano o Toscana\n,Location_\nAOC Bordeaux Superieur Rouge\n,Location_\nAOC Bourgogne Epineuil\n,Location_\nAOC Bourgogne Rouge\n,Location_\nAOC Bourgogne Tonnerre\n,Location_\nAOC Bourgueil\n,Location_\nAOC Brouilly\n,Location_\nAOC Chassagne-Montrachet\n,Location_\nAOC Chinon\n,Location_\nAOC Coteaux du Loir\n,Location_\nAOC Cotes de Provence Rouge\n,Location_\nAOC Crozes-Hermitage\n,Location_\nAOC Listrac-Médoc\n,Location_\nAOC Madiran Rouge\n,Location_\nAOC Morgon\n,Location_\nAOC Pessac Léognan\n,Location_\nAOC Rully Rouge\n,Location_\nAOC Saint Joseph\n,Location_\nAOC Vin de Savoie Arbin\n,Location_\nDO Bierzo\n,Location_\nDO Maipo\n,Location_\nDO Rias Baixas\n,Location_\nDO Ribeira Sacra\n,Location_\nDO Ribera del Duero\n,Location_\nDOC Aglianico del Vulture\n,Location_\nDOC Alghero Cabernet Riserva\n,Location_\nDOC Alto Adige Lagrein Riserva\n,Location_\nDOC Alto Adige Lagrein\n,...,Region_Bordeaux,Region_Bourgogne,Region_Burgenland,Region_Calabria,Region_Campania,Region_Casablanca Valley,Region_Castilla and Leon,Region_Catalunya,Region_Colchagua Valley,Region_Emilia-Romagna,Region_Friuli-Venezia Giulia,Region_Galicia,Region_Languedoc-Roussillon,Region_Lazio,Region_Liguria,Region_Loire,Region_Lombardy,Region_Maipo Valley,Region_Marche,Region_Mendoza,Region_Molise,Region_Oregon,Region_Patagonia,Region_Piedmont,Region_Provence,Region_Rapel Valley,Region_Rhône,Region_Rioja,Region_Sardinia,Region_Savoie,Region_Sicily,Region_South Africa,Region_South Australia,Region_Southwest,Region_Trentino Alto Adige,Region_Tuscany,Region_Umbria,Region_Veneto,"Volume_0,75 l Standard","Volume_1,50 l Magnum"
1,13.0,11.31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,13.5,10.91,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,13.0,11.96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
4,13.5,16.47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
5,13.0,9.04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
969,11.5,13.14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
971,12.5,14.19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
972,13.5,17.36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
973,12.5,8.59,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0


In [53]:
y = X.pop('Price')
y = pd.DataFrame(y)
y

,Price
1,11.31
2,10.91
3,11.96
4,16.47
5,9.04
...,...
969,13.14
971,14.19
972,17.36
973,8.59


In [0]:
#splitting
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(e.X, y, test_size=0.1, random_state=42)

In [55]:
y_test

,Price
232,54.07
958,35.29
88,12.75
76,11.98
918,54.90
...,...
39,39.27
37,10.44
611,28.58
320,14.60


In [56]:
#SVR, manual procedure
from sklearn.svm import SVR
clf = SVR(kernel='poly', C=1, gamma='auto', degree=3, epsilon=.1, coef0=1)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test) #???
from sklearn.metrics import r2_score
#r2 score
print(r2_score(y_test, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9999297145102094


In [57]:
#performance evalutation
v.cross_validation(clf, e.X, y, 10)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

('Accuracy: 1.00 (+/- 0.00)',
 array([0.99990355, 0.99996721, 0.99996876, 0.99994828, 0.99996389,
        0.99998166, 0.99996577, 0.99994188, 0.99988042, 0.99996255]))

In [58]:
#comparing results
y_test = y_test.reset_index()
y_test.pop('index')
y_test

y_pred = pd.DataFrame(y_pred)
total = pd.concat([y_test, y_pred], axis=1)
total

,Price,0
0,54.07,54.162864
1,35.29,35.135481
2,12.75,12.764813
3,11.98,12.002577
4,54.90,54.956669
...,...,...
79,39.27,39.428288
80,10.44,10.504983
81,28.58,28.523100
82,14.60,14.558757
